In [1]:
import openseespy.opensees as ops
import opsvis as opsv
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import pandas as pd

In [2]:
KK = []
for f_dof in range(1, 7):
    ## Create a frame model that is equivalent to STR-1
    beam_to_node = {}
    # Clear existing model
    ops.wipe()

    # Define Model Builder
    ops.model('basic', '-ndm', 3, '-ndf', 6)  # 3D model with 6 DOF per node

    # Define parameters
    n_bay = 1        # Number of bays
    l_bay = 1      # Length of each bay
    h_bay = 0.5      # Height of each story
    numStories = 1  # Number of stories

    # Material properties frame
    b_frame = 1
    d_beam = 0.12
    d_column = 0.12
    E = 70e9  # Young's Modulus in Pa
    E_column = E * 10000
    A_beam = b_frame * d_beam   # Area of the element in m^2
    A_column = b_frame * d_column
    mu = 0.28
    G = E/(2*(1+mu))
    G_column = E_column/(2*(1+mu))
    Iy_beam = b_frame * d_beam**3/12
    Iz_beam = d_beam**3 * b_frame/12  # Moment of Inertia in m^4
    Iy_column = b_frame * d_column**3/12
    Iz_column = d_column**3 * b_frame/12  # Moment of Inertia in m^4

    # calculate torsional constant for beam
    a = max(b_frame, d_beam)
    b = min(b_frame, d_beam)
    J_beam = a * b**3 / 16 * (16/3 - 3.36 * b/a * (1 - b**4/(12*a**4)))
    # calculate torsional constant for column
    a = max(b_frame, d_column)
    b = min(b_frame, d_column)
    J_column = a * b**3 / 16 * (16/3 - 3.36 * b/a * (1 - b**4/(12*a**4)))

    # Define footing meshing parameters
    num_nodes_in_frame = (n_bay + 1) * (numStories + 1)
    num_elems_in_frame = n_bay * numStories + (n_bay + 1) * numStories

    footing_nodes_ind = []
    footing_coord_x = []
    footing_coord_y = []
    footing_coord_z = []

    beam_nodes_ind = []
    beam_nodes_coord_x = []

    # Create nodes
    for i in range(numStories + 1):
        for j in range(n_bay + 1):
            nodeTag = i * (n_bay + 1) + j + 0
            x = j * l_bay
            z = i * h_bay
            ops.node(nodeTag, x, 0, z)
            if i == 0:
                footing_nodes_ind.append(nodeTag)
                footing_coord_x.append(x)
                footing_coord_y.append(0)
                footing_coord_z.append(z)
            if i == 1:
                beam_nodes_ind.append(nodeTag)
                beam_nodes_coord_x.append(x)
    beam_nodes_ind = np.array(beam_nodes_ind).astype(int)
    beam_nodes_coord_x = np.array(beam_nodes_coord_x).astype(float)
    # Define geometric transformation
    horizontal_gTTag = 1
    vertical_gTTag = 2
    ops.geomTransf('Linear', horizontal_gTTag, 0, 0, 1)
    ops.geomTransf('Linear', vertical_gTTag, 0, 1, 0)
    # Define elements
    for i in range(numStories):
        for j in range(n_bay):
            # Horizontal elements (beams)
            nodeI = (i + 1) * (n_bay + 1) + j + 0
            nodeJ = nodeI + 1
            eleTag = i * (n_bay) + j + 0
            ops.element('elasticBeamColumn', eleTag, nodeI, nodeJ, A_beam, E, G, J_beam, Iy_beam, Iz_beam, horizontal_gTTag)
            beam_to_node[eleTag] = {'nodes':[nodeI, nodeJ],
                                    'length': l_bay}

        for j in range(n_bay + 1):
            if i < numStories:
                # Vertical elements (columns)
                nodeI = i * (n_bay + 1) + j + 0
                nodeJ = nodeI + (n_bay + 1)
                eleTag = n_bay * numStories + i * (n_bay + 1) + j + 0
                ops.element('elasticBeamColumn', eleTag, nodeI, nodeJ, A_column, E_column, G_column, J_column, Iy_column, Iz_column, vertical_gTTag)


    footing_nodes_ind = np.array(footing_nodes_ind).astype(int)
    footing_coord_x = np.array(footing_coord_x).astype(float)
    footing_coord_y = np.array(footing_coord_y).astype(float)
    footing_coord_z = np.array(footing_coord_z).astype(float)

    # Calculate the self weight loads (simplified as point loads, needs refinement)
    num_of_nodes = num_nodes_in_frame + len(footing_nodes_ind) - (n_bay + 1)
    self_weight_unit = np.zeros(num_of_nodes * 6).astype(float)
    for _, nodes in beam_to_node.items():
        nodeI = nodes['nodes'][0]
        nodeJ = nodes['nodes'][1]
        length = nodes['length']
        self_weight_unit[nodeI * 6 + 2] += (-1/2*length)
        self_weight_unit[nodeJ * 6 + 2] += (-1/2*length)


    if f_dof == 1:
        ops.fix(0, 0, 1, 1, 1, 1, 1)  # Fix the first node 
        ops.fix(1, 1, 1, 1, 1, 1, 1)  # Fix the second node 
    elif f_dof == 2:
        ops.fix(0, 1, 1, 0, 1, 1, 1)  # Fix the first node 
        ops.fix(1, 1, 1, 1, 1, 1, 1)  # Fix the second node 
    elif f_dof == 3:
        ops.fix(0, 1, 1, 1, 1, 0, 1)  # Fix the first node 
        ops.fix(1, 1, 1, 1, 1, 1, 1)  # Fix the second node 
    elif f_dof == 4:
        ops.fix(0, 1, 1, 1, 1, 1, 1)  # Fix the first node 
        ops.fix(1, 0, 1, 1, 1, 1, 1)  # Fix the second node 
    elif f_dof == 5:
        ops.fix(0, 1, 1, 1, 1, 1, 1)  # Fix the first node 
        ops.fix(1, 1, 1, 0, 1, 1, 1)  # Fix the second node 
    elif f_dof == 6:
        ops.fix(0, 1, 1, 1, 1, 1, 1)  # Fix the first node 
        ops.fix(1, 1, 1, 1, 1, 0, 1)  # Fix the second node 

    # Define the time series
    ops.timeSeries('Linear', 1)
    ops.pattern('Plain', 1, 1)

    # Define the constraint method
    ops.constraints("Plain")


    if f_dof == 1:
        ops.load(0, 1, 0, 0, 0, 0, 0)
    elif f_dof == 2:
        ops.load(0, 0, 0, 1, 0, 0, 0)
    elif f_dof == 3:
        ops.load(0, 0, 0, 0, 0, 1, 0)
    elif f_dof == 4:
        ops.load(1, 1, 0, 0, 0, 0, 0)
    elif f_dof == 5:
        ops.load(1, 0, 0, 1, 0, 0, 0)
    elif f_dof == 6:
        ops.load(1, 0, 0, 0, 0, 1, 0)

    ops.system("FullGeneral")
    ops.numberer("Plain")
    ops.constraints("Plain")
    ops.integrator("LoadControl", 1.0)
    ops.algorithm("Linear")
    ops.analysis("Static")

    ops.analyze(1)
    ops.reactions()

    if f_dof == 1:
        K = np.array([1, ops.nodeReaction(0, 3), ops.nodeReaction(0, 5),
        ops.nodeReaction(1, 1), ops.nodeReaction(1, 3), ops.nodeReaction(1, 5)]) / ops.nodeDisp(0, 1)
    elif f_dof == 2:
        K = np.array([ops.nodeReaction(0, 1), 1, ops.nodeReaction(0, 5),
        ops.nodeReaction(1, 1), ops.nodeReaction(1, 3), ops.nodeReaction(1, 5)]) / ops.nodeDisp(0, 3)
    elif f_dof == 3:
        K = np.array([ops.nodeReaction(0, 1), ops.nodeReaction(0, 3), 1,
        ops.nodeReaction(1, 1), ops.nodeReaction(1, 3), ops.nodeReaction(1, 5)]) / ops.nodeDisp(0, 5)
    elif f_dof == 4:
        K = np.array([ops.nodeReaction(0, 1), ops.nodeReaction(0, 3), ops.nodeReaction(0, 5),
        1, ops.nodeReaction(1, 3), ops.nodeReaction(1, 5)]) / ops.nodeDisp(1, 1)
    elif f_dof == 5:
        K = np.array([ops.nodeReaction(0, 1), ops.nodeReaction(0, 3), ops.nodeReaction(0, 5),
        ops.nodeReaction(1, 1), 1, ops.nodeReaction(1, 5)]) / ops.nodeDisp(1, 3)
    elif f_dof == 6:
        K = np.array([ops.nodeReaction(0, 1), ops.nodeReaction(0, 3), ops.nodeReaction(0, 5),
        ops.nodeReaction(1, 1), ops.nodeReaction(1, 3), 1]) / ops.nodeDisp(1, 5)
    KK.append(K)
    # opsv.plot_model(element_labels=1, fig_wi_he = (10,10), local_axes=0,
    #                 node_labels=1);

In [3]:
KK = np.array(KK)
KK[abs(KK) < 1] = 0

In [4]:
KK[abs(KK) < 1] = 0
KK = KK.reshape(6, 6)

df = pd.DataFrame(KK)

# Display the DataFrame in a Jupyter notebook
df

,0,1,2,3,4,5
0,8.342073e+09,0.000000e+00,4.170828e+09,-8.342073e+09,0.000000e+00,-4.170828e+09
1,0.000000e+00,1.209235e+08,-6.046177e+07,0.000000e+00,-1.209235e+08,-6.046177e+07
2,4.170828e+09,-6.046177e+07,2.125620e+09,-4.170828e+09,6.046177e+07,-2.065158e+09
3,-8.342073e+09,0.000000e+00,-4.170828e+09,8.342073e+09,0.000000e+00,4.170828e+09
4,0.000000e+00,-1.209235e+08,6.046177e+07,0.000000e+00,1.209235e+08,6.046177e+07
5,-4.170828e+09,-6.046177e+07,-2.065158e+09,4.170828e+09,6.046177e+07,2.125620e+09


In [17]:
KK[5, 5] / (E * b_frame * d_beam / l_bay * h_bay**2 + 4 * E * Iz_beam / l_bay )

0.9931317155958181

In [16]:
KK[2, 5] - 2 * E * Iz_beam / l_bay

-2085317899.121102